In [ ]:
import pandas as pd
import warnings
import tkinter as tk
import os
from tkinter import messagebox
from datetime import datetime

warnings.filterwarnings("ignore")

hoy = datetime.now()
año = hoy.year
mes = hoy.month

mes_nombre = hoy.strftime('%b').upper()[:3] # ENE
mes_año = f"{mes_nombre}{str(año)[2:]}" # ENE24
fecha = f"{año}{str(mes).zfill(2)}" # 202401

asignacion_camp_path = f"bases/asignacion/{fecha}/base_asignacion_{mes_año}_CAMP.xlsx"
nombre_base_impago = f"BASE_CONDONACION_{fecha}.xlsx"
base_impago_path = f"bases/campañas/{fecha}/{nombre_base_impago}"

campañas_path = f"bases/campañas/{fecha}/CAMPAÑAS_{mes_año}.xlsx"
final_path = os.path.abspath(campañas_path)

print("Base asignacion:", asignacion_camp_path)
print("Base impago:", base_impago_path)
print("Campañas:", campañas_path)

In [25]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [26]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", "¿Cargar base asignacion?", icon='warning')
if result == 'yes':
    df_asignacion = pd.read_excel(asignacion_camp_path)

root.destroy()

In [ ]:
df_asignacion_test = df_asignacion.copy()
print(df_asignacion_test.columns)
print("Base Asignacion:", df_asignacion_test.shape)

df_asignacion_test.columns = clean_columns(df_asignacion_test.columns)
cols_required = ['CODIGO', 'CONT_18', 'CARTERA', 'NOMBRE', 'MONEDA', 'CAPITAL', 'CAMP', 'CAMPAÑA', 'TIPO_CARTERA', 'AGENCIA', 'TIPO_FONDO', 'OFICINA', 'TERRITORIO', 'ID_VTA']
df_asignacion_test = df_asignacion_test[cols_required]
print("Base Asignacion:", df_asignacion_test.shape)

df_asignacion_test.rename(columns={
    'CODIGO': 'CODIGO CENTRAL', 
    'CONT_18': 'N° CONTRATO', 
    'NOMBRE': 'NOMBRE DEL CLIENTE', 
    'CAPITAL': 'DEUDA CAPITAL', 
    'CAMP': '% CONDONACIÓN', 
    'CAMPAÑA': 'MONTO A PAGAR', 
    'TIPO_CARTERA': 'TIPO CARTERA', 
    'AGENCIA': 'AGENCIA DE COBRANZA', 
    'TIPO_FONDO': 'TIPO DE FONDO', 
    'ID_VTA': 'ID_VENTA'
}, inplace=True)

df_asignacion_test = df_asignacion_test[df_asignacion_test['TIPO CARTERA'] == 'UNSECURED']
df_asignacion_test = df_asignacion_test[df_asignacion_test['ID_VENTA'].isin(['-'])]
df_asignacion_test = df_asignacion_test[~df_asignacion_test['AGENCIA DE COBRANZA'].isin(['SIN AGENCIA'])]

df_asignacion_test['CODIGO CENTRAL'] = df_asignacion_test['CODIGO CENTRAL'].astype('Int64').astype(str).str.zfill(8)
df_asignacion_test['N° CONTRATO'] = df_asignacion_test['N° CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
df_asignacion_test['MONTO A CONDONAR'] = df_asignacion_test['DEUDA CAPITAL'] * df_asignacion_test['% CONDONACIÓN']
df_asignacion_test['TIPO DE FONDO'] = df_asignacion_test['TIPO DE FONDO'].fillna('NULL')
df_asignacion_test['MONTO A PAGAR'] = df_asignacion_test.apply(lambda x: "DEUDA TOTAL" if x['TIPO DE FONDO'] in ['REACTIVA', 'FAE', 'CRECER'] else x['MONTO A PAGAR'], axis=1)

correos = {
    ('ASESCOM RJ', 'BC'): 'karol.lopez@rjabogados.com',
    ('ASESCOM RJ', 'EXJ'): 'lilia.gonzales@rjabogados.com',
    ('ASESCOM RJ', 'KSTBC'): 'rosa.hoyos@rjabogados.com',
    ('CLASA MORA', 'BC'): 'jsalazar@clasaperu.com',
    ('CLASA MORA', 'EXJ'): 'jayra@clasaperu.com',
    ('CLASA MORA', 'KSTBC'): 'jsolis@clasaperu.com',
    ('MORNESE MORA', 'BC'): 'brenda.paz@mornese.pe, juan.mendoza@mornese.pe, informes@mornese.pe',
    ('MORNESE MORA', 'EXJ'): 'aparco@mornese.pe, juan.mendoza@mornese.pe, informes@mornese.pe',
    ('MORNESE MORA', 'KSTBC'): 'juan.mendoza@mornese.pe, geraldine.hinojosa@mornese.pe, informes@mornese.pe',
    ('Muñiz', None): 'szegarra@munizlaw.com',
    ('RodriguezAngobaldo', None): 'ysifuentes@er.com.pe'
}

telefonos = {
    ('ASESCOM RJ', 'BC'): '955013048, 932811608',
    ('ASESCOM RJ', 'EXJ'): '932812578, 932811752',
    ('ASESCOM RJ', 'KSTBC'): '979270273, 932810247, 945309798',
    ('CLASA MORA', 'BC'): '940125238, 977746384',
    ('CLASA MORA', 'EXJ'): '940619571, 977746384',
    ('CLASA MORA', 'KSTBC'): '936994661, 977746384',
    ('MORNESE MORA', 'BC'): '017057808, 953090160',
    ('MORNESE MORA', 'EXJ'): '01-7057893, 977288425',
    ('MORNESE MORA', 'KSTBC'): '017057807, 967140068, 994376233',
    ('Muñiz', None): '999868827',
    ('RodriguezAngobaldo', None): '924104114'
}

def asignar_correo(row):
    return correos.get((row['AGENCIA DE COBRANZA'], row['CARTERA']), correos.get((row['AGENCIA DE COBRANZA'], None), None))

def asignar_telefono(row):
    return telefonos.get((row['AGENCIA DE COBRANZA'], row['CARTERA']), telefonos.get((row['AGENCIA DE COBRANZA'], None), None))

df_asignacion_test['CORREO DE LA AGENCIA'] = df_asignacion_test.apply(asignar_correo, axis=1).str.upper()
df_asignacion_test['TELÉFONO DE AGENCIA'] = df_asignacion_test.apply(asignar_telefono, axis=1).str.upper()
df_asignacion_test['AGENCIA DE COBRANZA'] = df_asignacion_test['AGENCIA DE COBRANZA'].str.upper()

df_asignacion_test['CONTACTO BBVA'] = 'recuperaciones@opplus.bbva.com'.upper()
df_asignacion_test['CORREO BBVAA'] = 'recuperaciones@opplus.bbva.com'.upper()
df_asignacion_test['TELÉFONO'] = '595-0030'

df_asignacion_test.sort_values(by=['CARTERA', 'CODIGO CENTRAL', '% CONDONACIÓN'], inplace=True)
df_asignacion_test.reset_index(drop=True, inplace=True)

cols_final = ['CODIGO CENTRAL', 'N° CONTRATO', 'CARTERA', 'NOMBRE DEL CLIENTE', 'MONEDA', 'DEUDA CAPITAL', '% CONDONACIÓN', 'MONTO A CONDONAR', 'MONTO A PAGAR', 'AGENCIA DE COBRANZA', 'TIPO DE FONDO', 'TELÉFONO DE AGENCIA', 'CORREO DE LA AGENCIA', 'CONTACTO BBVA', 'CORREO BBVAA' , 'TELÉFONO', 'OFICINA', 'TERRITORIO']
df_asignacion_test = df_asignacion_test[cols_final]

print("Campaña Asignación:", df_asignacion_test.shape)
df_asignacion_test.head(5)

In [28]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", "¿Cargar base impago?", icon='warning')
if result == 'yes':
    df_impago = pd.read_excel(base_impago_path, sheet_name='DATA')
    
    df_impago_test = df_impago.copy()
    print(df_impago_test.columns)
    print("Base Impago:", df_impago_test.shape)
    
    df_impago_test.columns = clean_columns(df_impago_test.columns)
    cols_required = ['CODCENTRAL', 'CONTRATO', 'CLIENTE', 'MONEDA', 'DIRECTA', 'TASA_DSCTO', 'MONTOPAGO', 'ENTIDAD_COB', 'OFICINA', 'TERRITORIO', 'SEGMENTO']
    df_impago_test = df_impago_test[cols_required]
    print("Base Impago:", df_impago_test.shape)
    
    df_impago_test.rename(columns={
        'CODCENTRAL': 'CODIGO CENTRAL', 
        'CONTRATO': 'N° CONTRATO', 
        'CLIENTE': 'NOMBRE DEL CLIENTE', 
        'DIRECTA': 'DEUDA CAPITAL', 
        'TASA_DSCTO': '% CONDONACIÓN', 
        'MONTOPAGO': 'MONTO A PAGAR', 
        'ENTIDAD_COB': 'AGENCIA DE COBRANZA', 
    }, inplace=True)
    
    df_impago_test['CARTERA'] = 'IMPAGO'
    df_impago_test['TIPO DE FONDO'] = 'NULL'
    
    df_impago_test['CODIGO CENTRAL'] = df_impago_test['CODIGO CENTRAL'].astype(str).astype('Int64').astype(str).str.zfill(8)
    df_impago_test['N° CONTRATO'] = df_impago_test['N° CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
    df_impago_test['MONTO A CONDONAR'] = df_impago_test['DEUDA CAPITAL'] * df_impago_test['% CONDONACIÓN']
    df_impago_test['TERRITORIO'] = df_impago_test['TERRITORIO'].fillna('NULL').astype(str)
    
    correos = {
        ('RECSA', 'PARTICULARES'): 'alfonso.moises@recsa.com / jean.yanqui@recsa.com',
        ('RECSA', 'PARTICULARES CONVENIOS'): 'alfonso.moises@recsa.com / jean.yanqui@recsa.com',
        ('RECSA', 'PYME'): 'sandra.clavo@recsa.com / katiuska.vera@recsa.com',
        ('ATENTO', 'PARTICULARES'): 'cvillarj@atento.com',
        ('ATENTO', 'PARTICULARES CONVENIOS'): 'cvillarj@atento.com',
        ('ATENTO', 'PYME'): 'jtumialanp@atento.com',
        ('ASESCOM RJ', 'PARTICULARES'): 'virginia.sifuentes@rjabogados.com',
        ('ASESCOM RJ', 'PARTICULARES CONVENIOS'): 'virginia.sifuentes@rjabogados.com',
        ('ASESCOM RJ', 'PYME'): 'bety.cabrera@rjabogados.com',
    }
    
    telefonos = {
        ('RECSA', 'PARTICULARES'): '01 7010519 / 961370088',
        ('RECSA', 'PARTICULARES CONVENIOS'): '01 7010519 / 961370088',
        ('RECSA', 'PYME'): '01 7099302',
        ('ATENTO', 'PARTICULARES'): '996044852 / 990384855 / 967683768',
        ('ATENTO', 'PARTICULARES CONVENIOS'): '996044852 / 990384855 / 967683768',
        ('ATENTO', 'PYME'): '987708366',
        ('ASESCOM RJ', 'PARTICULARES'): '938562012',
        ('ASESCOM RJ', 'PARTICULARES CONVENIOS'): '938562012',
        ('ASESCOM RJ', 'PYME'): '908874634',
    }
    
    def asignar_correo(row):
        return correos.get((row['AGENCIA DE COBRANZA'], row['SEGMENTO']), correos.get((row['AGENCIA DE COBRANZA'], None), None))
    
    def asignar_telefono(row):
        return telefonos.get((row['AGENCIA DE COBRANZA'], row['SEGMENTO']), telefonos.get((row['AGENCIA DE COBRANZA'], None), None))
    
    df_impago_test['CORREO DE LA AGENCIA'] = df_impago_test.apply(asignar_correo, axis=1)
    df_impago_test['TELÉFONO DE AGENCIA'] = df_impago_test.apply(asignar_telefono, axis=1)
    
    df_impago_test['CONTACTO BBVA'] = 'recuperaciones@opplus.bbva.com'
    df_impago_test['CORREO BBVAA'] = 'recuperaciones@opplus.bbva.com'
    df_impago_test['TELÉFONO'] = '595-0030'
    
    df_impago_test.sort_values(by=['CARTERA', 'CODIGO CENTRAL', '% CONDONACIÓN'], inplace=True)
    df_impago_test.reset_index(drop=True, inplace=True)
    
    cols_final = ['CODIGO CENTRAL', 'N° CONTRATO', 'CARTERA', 'NOMBRE DEL CLIENTE', 'MONEDA', 'DEUDA CAPITAL', '% CONDONACIÓN', 'MONTO A CONDONAR', 'MONTO A PAGAR', 'AGENCIA DE COBRANZA', 'TIPO DE FONDO', 'TELÉFONO DE AGENCIA', 'CORREO DE LA AGENCIA', 'CONTACTO BBVA', 'CORREO BBVAA' , 'TELÉFONO', 'OFICINA', 'TERRITORIO']
    df_impago_test = df_impago_test[cols_final]
    
    print("Campaña Impago:", df_impago_test.shape)
    
    df_camp_final = pd.concat([df_asignacion_test, df_impago_test])
    print("Campaña total:", df_camp_final.shape)
    
    df_camp_final['CODIGO CENTRAL'] = df_camp_final['CODIGO CENTRAL'].astype(str).astype("Int64").astype(str).str.zfill(8)
    df_camp_final['N° CONTRATO'] = df_camp_final['N° CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
    
    df_camp_final.to_excel(campañas_path, index=False, sheet_name='BASE')
else:
    df_asignacion_test.to_excel(campañas_path, index=False, sheet_name='BASE')

result = messagebox.askquestion("Confirmación", "¿Abrir archivo?", icon='warning')
if result == 'yes':
    os.startfile(final_path)

root.destroy()